<a href="https://colab.research.google.com/github/meghana-0211/Credit-Card-Behaviour/blob/main/ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.base import clone
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import warnings
import zipfile
import os
import tempfile
warnings.filterwarnings('ignore')

def extract_zip(zip_path, extract_dir=None):
    """
    Extract a ZIP file and return the path to the CSV file inside.

    Parameters:
    zip_path (str): Path to the ZIP file
    extract_dir (str): Directory to extract files to. If None, uses a temporary directory

    Returns:
    str: Path to the extracted CSV file
    """
    if extract_dir is None:
        extract_dir = tempfile.mkdtemp()

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

        # Find the CSV file in the extracted contents
        csv_files = [f for f in zip_ref.namelist() if f.endswith('.csv')]
        if not csv_files:
            raise ValueError("No CSV file found in the ZIP archive")

        return os.path.join(extract_dir, csv_files[0])

class BalancedEnsembleCreditScorer:
    def __init__(self, random_state=42):
        self.random_state = random_state
        self.scaler = StandardScaler()
        self.feature_names_ = None
        self.training_features_ = None

        # Define base models with provided parameters
        self.base_models = {
            'xgb1': xgb.XGBClassifier(
                n_estimators=500, learning_rate=0.02, max_depth=4,
                subsample=0.8, colsample_bytree=0.8, min_child_weight=4,
                scale_pos_weight=3, random_state=random_state
            ),
            'xgb2': xgb.XGBClassifier(
                n_estimators=500, learning_rate=0.03, max_depth=5,
                subsample=0.85, colsample_bytree=0.9, min_child_weight=6,
                scale_pos_weight=3, random_state=random_state
            ),
            'xgb3': xgb.XGBClassifier(
                n_estimators=500, learning_rate=0.05, max_depth=3,
                subsample=0.9, colsample_bytree=0.75, min_child_weight=3,
                scale_pos_weight=3, random_state=random_state
            ),
            'rf1': RandomForestClassifier(
                n_estimators=500, max_depth=5, min_samples_split=25,
                min_samples_leaf=10, class_weight={0: 1, 1: 3},
                random_state=random_state
            ),
            'rf2': RandomForestClassifier(
                n_estimators=500, max_depth=6, min_samples_split=20,
                min_samples_leaf=8, class_weight={0: 1, 1: 3},
                random_state=random_state
            ),
            'rf3': RandomForestClassifier(
                n_estimators=500, max_depth=4, min_samples_split=30,
                min_samples_leaf=12, class_weight={0: 1, 1: 3},
                random_state=random_state
            )
        }

        # Initialize meta-model
        self.meta_model = LogisticRegression(
            C=0.8,
            class_weight='balanced',
            random_state=random_state,
            max_iter=1000
        )

        # Initialize containers for trained models
        self.trained_models = {i: {} for i in range(1, 7)}  # One dict per balanced dataset

    def engineer_features(self, X):
        """Engineer features from transaction, bureau, and onus attributes"""
        X_new = X.copy()

        # Transaction attribute aggregations
        tx_cols = [col for col in X.columns if col.startswith('transaction_attribute_')]
        if tx_cols:
            X_new['avg_nonzero_tx'] = X[tx_cols].replace(0, np.nan).mean(axis=1)
            X_new['nonzero_tx_count'] = (X[tx_cols] != 0).sum(axis=1)
            X_new['tx_volatility'] = X[tx_cols].std(axis=1)

        # Bureau data aggregations
        bureau_cols = [col for col in X.columns if col.startswith('bureau_')]
        if bureau_cols:
            X_new['avg_bureau_score'] = X[bureau_cols].mean(axis=1)
            X_new['bureau_score_spread'] = X[bureau_cols].std(axis=1)
            X_new['active_bureau_count'] = (X[bureau_cols] != 0).sum(axis=1)

        # Onus attribute aggregations
        onus_cols = [col for col in X.columns if col.startswith('onus_attribute_')]
        if onus_cols:
            X_new['avg_onus_score'] = X[onus_cols].mean(axis=1)
            X_new['onus_volatility'] = X[onus_cols].std(axis=1)

        # Bureau enquiry features
        enquiry_cols = [col for col in X.columns if col.startswith('bureau_enquiry_')]
        if enquiry_cols:
            X_new['total_enquiries'] = X[enquiry_cols].sum(axis=1)
            recent_enquiry_cols = enquiry_cols[-10:]
            X_new['recent_enquiry_intensity'] = X[recent_enquiry_cols].sum(axis=1)

        # Interaction features
        if 'avg_bureau_score' in X_new.columns and 'total_enquiries' in X_new.columns:
            X_new['bureau_enquiry_risk'] = X_new['avg_bureau_score'] / (X_new['total_enquiries'] + 1)

        # Fill NaN values and store feature names
        X_new = X_new.fillna(0)
        self.feature_names_ = list(X_new.columns)

        return X_new

    def _get_base_predictions(self, X_scaled):
        """Get predictions from all base models"""
        predictions = np.zeros((X_scaled.shape[0], len(self.trained_models) * len(self.base_models)))
        col_idx = 0

        for dataset_idx in self.trained_models:
            for model_name, model in self.trained_models[dataset_idx].items():
                predictions[:, col_idx] = model.predict_proba(X_scaled)[:, 1]
                col_idx += 1

        return predictions

    def fit(self, balanced_datasets):
        """
        Fit models on multiple balanced datasets and train meta-model

        Parameters:
        balanced_datasets: dict of pandas DataFrames, keys should be 1-6
        """
        print("Training base models on balanced datasets...")

        # Store feature names from first dataset
        X_first = balanced_datasets[1].drop(['bad_flag', 'account_number'], axis=1)
        X_engineered = self.engineer_features(X_first)
        self.training_features_ = self.feature_names_

        # Initialize array to store meta-features
        meta_features = []
        meta_labels = []

        # Train models on each balanced dataset
        for dataset_idx, df in balanced_datasets.items():
            print(f"\nTraining models on balanced dataset {dataset_idx}")

            # Prepare features
            X = df.drop(['bad_flag', 'account_number'], axis=1)
            y = df['bad_flag']

            # Engineer and scale features
            X_engineered = self.engineer_features(X)
            X_scaled = pd.DataFrame(
                self.scaler.fit_transform(X_engineered),
                columns=self.feature_names_,
                index=X_engineered.index
            )

            # Train each base model
            for model_name, model in self.base_models.items():
                print(f"Training {model_name} on dataset {dataset_idx}...")
                model_clone = clone(model)
                model_clone.fit(X_scaled, y)
                self.trained_models[dataset_idx][model_name] = model_clone

            # Get predictions for meta-model training
            dataset_predictions = self._get_base_predictions(X_scaled)
            meta_features.append(dataset_predictions)
            meta_labels.append(y)

        # Combine all meta-features and train meta-model
        print("\nTraining meta-model...")
        X_meta = np.vstack(meta_features)
        y_meta = np.concatenate(meta_labels)
        self.meta_model.fit(X_meta, y_meta)

        return self

    def predict_proba(self, X):
        """Generate predictions using meta-model combination of base models"""
        # Engineer features
        X_engineered = self.engineer_features(X)
        X_engineered = X_engineered[self.training_features_]

        # Scale features
        X_scaled = pd.DataFrame(
            self.scaler.transform(X_engineered),
            columns=self.training_features_,
            index=X_engineered.index
        )

        # Get base model predictions
        base_predictions = self._get_base_predictions(X_scaled)

        # Generate final predictions using meta-model
        final_predictions = self.meta_model.predict_proba(base_predictions)[:, 1]

        return final_predictions

    def evaluate(self, X, y):
        """Evaluate model performance with multiple metrics"""
        # Generate predictions
        y_pred_proba = self.predict_proba(X)

        # Find optimal threshold using precision-recall curve
        precisions, recalls, thresholds = precision_recall_curve(y, y_pred_proba)
        f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
        optimal_threshold = thresholds[np.argmax(f1_scores[:-1])]

        # Generate binary predictions using optimal threshold
        y_pred = (y_pred_proba >= optimal_threshold).astype(int)

        # Calculate metrics
        tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

        metrics = {
            'threshold': optimal_threshold,
            'accuracy': accuracy_score(y, y_pred),
            'auc_score': roc_auc_score(y, y_pred_proba),
            'avg_precision': average_precision_score(y, y_pred_proba),
            'confusion_matrix': {
                'tn': tn, 'fp': fp,
                'fn': fn, 'tp': tp
            }
        }

        # Print detailed metrics
        print("\nModel Performance Metrics:")
        print("=" * 50)
        print(f"Optimal Threshold: {optimal_threshold:.4f}")
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"AUC Score: {metrics['auc_score']:.4f}")
        print(f"Average Precision: {metrics['avg_precision']:.4f}")

        print("\nConfusion Matrix:")
        print("-" * 50)
        print(f"True Negatives: {tn}")
        print(f"False Positives: {fp}")
        print(f"False Negatives: {fn}")
        print(f"True Positives: {tp}")

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        print("\nAdditional Metrics:")
        print("-" * 50)
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")

        return metrics

In [16]:
# Example usage
if __name__ == "__main__":
    # Load and split balanced datasets
    balanced_datasets_train = {}
    balanced_datasets_test = {}

    print("Loading and splitting datasets...")
    for i in range(1, 7):
        # Load dataset
        df = pd.read_excel('/content/balanced_datasets (1).xlsx', sheet_name=f'df_{i}')

        # Split the data
        train_df, test_df = train_test_split(
            df, test_size=0.2, random_state=42,
            stratify=df['bad_flag']
        )

        balanced_datasets_train[i] = train_df
        balanced_datasets_test[i] = test_df

    # Initialize and train model
    print("\nTraining model...")
    model = BalancedEnsembleCreditScorer()
    model.fit(balanced_datasets_train)

    # Combine all test sets and remove duplicates
    print("\nPreparing test data...")
    all_test_data = pd.concat([df for df in balanced_datasets_test.values()], axis=0)
    all_test_data = all_test_data.drop_duplicates(subset='account_number')

    # Evaluate model
    print("\nEvaluating model on test set...")
    X_test = all_test_data.drop(['bad_flag', 'account_number'], axis=1)
    y_test = all_test_data['bad_flag']

    # Get evaluation metrics
    metrics = model.evaluate(X_test, y_test)

    # Generate and save predictions
    predictions = pd.DataFrame({
        'account_number': all_test_data['account_number'],
        'actual': y_test,
        'predicted_probability': model.predict_proba(X_test)
    })
    predictions.to_csv('predictions_balanced_ensemble101.csv', index=False)

Loading and splitting datasets...

Training model...
Training base models on balanced datasets...

Training models on balanced dataset 1
Training xgb1 on dataset 1...
Training xgb2 on dataset 1...
Training xgb3 on dataset 1...
Training rf1 on dataset 1...
Training rf2 on dataset 1...
Training rf3 on dataset 1...

Training models on balanced dataset 2
Training xgb1 on dataset 2...
Training xgb2 on dataset 2...
Training xgb3 on dataset 2...
Training rf1 on dataset 2...
Training rf2 on dataset 2...
Training rf3 on dataset 2...

Training models on balanced dataset 3
Training xgb1 on dataset 3...
Training xgb2 on dataset 3...
Training xgb3 on dataset 3...
Training rf1 on dataset 3...
Training rf2 on dataset 3...
Training rf3 on dataset 3...

Training models on balanced dataset 4
Training xgb1 on dataset 4...
Training xgb2 on dataset 4...
Training xgb3 on dataset 4...
Training rf1 on dataset 4...
Training rf2 on dataset 4...
Training rf3 on dataset 4...

Training models on balanced dataset 5

In [17]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tempfile

# Example usage
if __name__ == "__main__":
    # Create temporary directory for zip extraction
    temp_dir = tempfile.mkdtemp()

    # Load and split balanced datasets
    balanced_datasets_train = {}
    balanced_datasets_test = {}

    print("Loading and splitting datasets...")
    for i in range(1, 7):
        # Load dataset
        df = pd.read_excel('/content/balanced_datasets (1).xlsx', sheet_name=f'df_{i}')

        # Split the data
        train_df, test_df = train_test_split(
            df, test_size=0.2, random_state=42,
            stratify=df['bad_flag']
        )

        balanced_datasets_train[i] = train_df
        balanced_datasets_test[i] = test_df

    # Initialize and train model
    print("\nTraining model...")
    model = BalancedEnsembleCreditScorer()
    model.fit(balanced_datasets_train)

    # Combine all test sets and remove duplicates
    print("\nPreparing test data...")
    all_test_data = pd.concat([df for df in balanced_datasets_test.values()], axis=0)
    all_test_data = all_test_data.drop_duplicates(subset='account_number')

    # Evaluate model
    print("\nEvaluating model on test set...")
    X_test = all_test_data.drop(['bad_flag', 'account_number'], axis=1)
    y_test = all_test_data['bad_flag']

    # Get evaluation metrics
    metrics = model.evaluate(X_test, y_test)

    # Generate and save predictions for test set
    predictions = pd.DataFrame({
        'account_number': all_test_data['account_number'],
        'actual': y_test,
        'predicted_probability': model.predict_proba(X_test)
    })
    predictions.to_csv('predictions_balanced_ensemble101.csv', index=False)

    # Load and process validation data
    print("\nProcessing validation data...")
    val_csv_path = extract_zip('/content/validation_data_to_be_shared 3.zip', temp_dir)
    val_data = pd.read_csv(val_csv_path)

    # Generate predictions for validation data
    print("Generating predictions for validation data...")
    val_predictions = model.predict_proba(val_data.drop(['account_number'], axis=1))

    # Create submission file
    submission = pd.DataFrame({
        'account_number': val_data['account_number'],
        'predicted_probability': val_predictions
    })

    # Save validation predictions
    print("Saving validation predictions...")
    submission.to_csv('validation_predictions.csv', index=False)

    print("\nCompleted! Files saved:")
    print("1. predictions_balanced_ensemble101.csv - Test set predictions")
    print("2. validation_predictions.csv - Validation set predictions")

Loading and splitting datasets...

Training model...
Training base models on balanced datasets...

Training models on balanced dataset 1
Training xgb1 on dataset 1...
Training xgb2 on dataset 1...
Training xgb3 on dataset 1...
Training rf1 on dataset 1...
Training rf2 on dataset 1...
Training rf3 on dataset 1...

Training models on balanced dataset 2
Training xgb1 on dataset 2...
Training xgb2 on dataset 2...
Training xgb3 on dataset 2...
Training rf1 on dataset 2...
Training rf2 on dataset 2...
Training rf3 on dataset 2...

Training models on balanced dataset 3
Training xgb1 on dataset 3...
Training xgb2 on dataset 3...
Training xgb3 on dataset 3...
Training rf1 on dataset 3...
Training rf2 on dataset 3...
Training rf3 on dataset 3...

Training models on balanced dataset 4
Training xgb1 on dataset 4...
Training xgb2 on dataset 4...
Training xgb3 on dataset 4...
Training rf1 on dataset 4...
Training rf2 on dataset 4...
Training rf3 on dataset 4...

Training models on balanced dataset 5